### Imports

In [2]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv(r'data/gas_data_clean.csv')
df

,Geography,Nov-92,Dec-92,Jan-93,Feb-93,Mar-93,Apr-93,May-93,Jun-93,Jul-93,...,Apr-23,May-23,Jun-23,Jul-23,Aug-23,Sep-23,Oct-23,Nov-23,Dec-23,Jan-24
0,Canada,56.1,56.1,55.9,54.9,54.0,54.1,54.5,55.5,55.2,...,160.1,158.8,161.6,163.1,170.6,168.3,157.4,152.2,145.4,144.1
1,"St. John's, Newfoundland and Labrador",59.4,59.2,57.1,57.1,56.4,56.4,57.4,58.4,59.5,...,174.3,165.5,168.4,174.8,188.0,190.0,173.9,168.3,163.0,161.2
2,"Charlottetown and Summerside, Prince Edward Is...",60.9,60.9,58.9,58.9,58.9,56.4,55.9,55.6,56.2,...,168.4,161.2,163.4,173.7,185.3,184.8,165.7,161.3,158.0,156.6
3,"Halifax, Nova Scotia",57.3,56.0,52.2,50.9,49.2,49.6,51.3,51.8,53.6,...,163.2,151.4,153.1,174.7,185.0,182.2,166.3,161.6,154.3,154.5
4,"Saint John, New Brunswick",56.1,55.3,54.6,54.5,54.5,54.4,54.7,54.8,55.7,...,169.3,159.1,161.4,170.6,182.7,184.5,167.3,160.3,154.8,152.7
5,"Québec, Quebec",59.6,60.4,59.5,57.9,58.4,58.3,59.1,61.0,61.5,...,171.7,173.8,177.9,174.6,185.0,182.6,173.1,166.3,163.0,160.2
6,"Montréal, Quebec",59.5,60.7,58.8,57.6,58.1,57.5,58.7,61.4,59.8,...,167.4,166.4,169.1,172.2,179.8,178.4,166.2,162.0,158.2,158.5
7,"Ottawa-Gatineau, Ontario part, Ontario/Quebec 2",56.4,56.0,55.7,55.0,54.6,54.5,54.1,55.9,55.1,...,157.8,156.0,158.7,162.0,168.9,164.9,152.5,148.7,143.6,143.9
8,"Toronto, Ontario",55.5,52.5,53.1,52.7,50.9,51.3,52.0,53.4,50.0,...,156.6,156.1,159.1,162.5,169.4,165.6,152.9,148.8,143.7,144.2
9,"Thunder Bay, Ontario 3",58.0,57.9,57.9,57.5,56.4,55.7,56.5,58.5,57.5,...,165.7,162.6,167.0,161.8,170.0,168.2,154.2,153.0,137.3,136.8
